# Lab 03: Insurance Dataset Linear Regression Analysis

**Department of Electrical and Computer Engineering**  
**Pak-Austria Fachhochschule: Institute of Applied Sciences & Technology**  
**Subject: Machine Learning**  
**Subject Teacher: Dr. Abid Ali**  
**Lab Supervisor: Miss. Sana Saleem**

## Lab Tasks Objectives
1. Implement linear regression to predict insurance charges using a real-world dataset
2. Explore and preprocess datasets, including handling missing values and outliers
3. Apply data visualization techniques to understand data distributions and correlations
4. Train and evaluate a linear regression model using performance metrics
5. Use gradient descent to optimize model parameters and understand the cost function
6. Analyze model performance using metrics such as accuracy, precision, recall, and F1-score
7. Visualize the importance of features in the model and analyze training loss over time

## Dataset
- **File**: Insurance.csv
- **Features**: age, gender, bmi, children, smoker, region
- **Target Variable**: charges (medical insurance charges)
- **Total Records**: 1,338


In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (mean_squared_error, r2_score, mean_absolute_error,
                           accuracy_score, precision_score, recall_score, f1_score,
                           confusion_matrix, roc_curve, auc, precision_recall_curve)
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")

print("All libraries imported successfully!")


## Task 1: Data Loading and Exploration


In [ ]:
# Load the insurance dataset
df = pd.read_csv('Insurance.csv')

print("INSURANCE DATASET ANALYSIS")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Features: {list(df.columns)}")
print(f"Data types:\n{df.dtypes}")

print("\nFirst 5 rows:")
print(df.head())

print("\nDataset Statistics:")
print(df.describe())

print("\nMissing Values:")
print(df.isnull().sum())

print(f"\nMissing Values Percentage:")
print((df.isnull().sum() / len(df)) * 100)


## Task 2: Data Visualization and Analysis


In [ ]:
# Comprehensive data visualization
fig = plt.figure(figsize=(20, 16))

# 1. Target variable distribution
plt.subplot(4, 4, 1)
plt.hist(df['charges'], bins=30, alpha=0.7, color='steelblue', edgecolor='black')
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges ($)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 2. Log-transformed charges
plt.subplot(4, 4, 2)
plt.hist(np.log(df['charges']), bins=30, alpha=0.7, color='crimson', edgecolor='black')
plt.title('Log-Transformed Charges')
plt.xlabel('Log(Charges)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 3. Age distribution
plt.subplot(4, 4, 3)
plt.hist(df['age'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 4. BMI distribution
plt.subplot(4, 4, 4)
plt.hist(df['bmi'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
plt.title('BMI Distribution')
plt.xlabel('BMI')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# 5. Gender distribution
plt.subplot(4, 4, 5)
gender_counts = df['gender'].value_counts()
plt.pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%', 
        colors=['lightcoral', 'lightblue'], startangle=90)
plt.title('Gender Distribution')

# 6. Smoking status
plt.subplot(4, 4, 6)
smoker_counts = df['smoker'].value_counts()
plt.bar(smoker_counts.index, smoker_counts.values, color=['lightgreen', 'salmon'])
plt.title('Smoking Status Distribution')
plt.xlabel('Smoking Status')
plt.ylabel('Count')

# 7. Region distribution
plt.subplot(4, 4, 7)
region_counts = df['region'].value_counts()
plt.bar(region_counts.index, region_counts.values, color='lightsteelblue')
plt.title('Geographic Region Distribution')
plt.xlabel('Region')
plt.ylabel('Count')
plt.xticks(rotation=45)

# 8. Children distribution
plt.subplot(4, 4, 8)
children_counts = df['children'].value_counts().sort_index()
plt.bar(children_counts.index, children_counts.values, color='gold')
plt.title('Number of Children Distribution')
plt.xlabel('Number of Children')
plt.ylabel('Count')

# 9. Age vs Charges
plt.subplot(4, 4, 9)
plt.scatter(df['age'], df['charges'], alpha=0.6, color='forestgreen')
plt.title('Age vs Insurance Charges')
plt.xlabel('Age')
plt.ylabel('Charges ($)')
plt.grid(True, alpha=0.3)

# 10. BMI vs Charges
plt.subplot(4, 4, 10)
plt.scatter(df['bmi'], df['charges'], alpha=0.6, color='purple')
plt.title('BMI vs Insurance Charges')
plt.xlabel('BMI')
plt.ylabel('Charges ($)')
plt.grid(True, alpha=0.3)

# 11. Charges by Smoking Status
plt.subplot(4, 4, 11)
df.boxplot(column='charges', by='smoker', ax=plt.gca())
plt.title('Charges Distribution by Smoking Status')
plt.suptitle('')
plt.xlabel('Smoking Status')
plt.ylabel('Charges ($)')
plt.grid(True, alpha=0.3)

# 12. Charges by Gender
plt.subplot(4, 4, 12)
df.boxplot(column='charges', by='gender', ax=plt.gca())
plt.title('Charges Distribution by Gender')
plt.suptitle('')
plt.xlabel('Gender')
plt.ylabel('Charges ($)')
plt.grid(True, alpha=0.3)

# 13. Correlation heatmap
plt.subplot(4, 4, 13)
# Encode categorical variables for correlation
df_corr = df.copy()
le_gender = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

df_corr['gender_encoded'] = le_gender.fit_transform(df_corr['gender'])
df_corr['smoker_encoded'] = le_smoker.fit_transform(df_corr['smoker'])
df_corr['region_encoded'] = le_region.fit_transform(df_corr['region'])

numerical_cols = ['age', 'bmi', 'children', 'gender_encoded', 'smoker_encoded', 'region_encoded', 'charges']
correlation_matrix = df_corr[numerical_cols].corr()

sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix')

# 14. Charges by Region
plt.subplot(4, 4, 14)
df.boxplot(column='charges', by='region', ax=plt.gca())
plt.title('Charges Distribution by Region')
plt.suptitle('')
plt.xlabel('Region')
plt.ylabel('Charges ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 15. Children vs Charges
plt.subplot(4, 4, 15)
df.boxplot(column='charges', by='children', ax=plt.gca())
plt.title('Charges Distribution by Number of Children')
plt.suptitle('')
plt.xlabel('Number of Children')
plt.ylabel('Charges ($)')
plt.grid(True, alpha=0.3)

# 16. 3D scatter plot (Age, BMI, Charges)
plt.subplot(4, 4, 16)
ax = plt.gca()
scatter = ax.scatter(df['age'], df['bmi'], c=df['charges'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Charges ($)')
plt.xlabel('Age')
plt.ylabel('BMI')
plt.title('Age vs BMI (colored by Charges)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\nCHARGES STATISTICAL SUMMARY")
print("=" * 40)
print(f"Mean: ${df['charges'].mean():.2f}")
print(f"Median: ${df['charges'].median():.2f}")
print(f"Standard Deviation: ${df['charges'].std():.2f}")
print(f"Minimum: ${df['charges'].min():.2f}")
print(f"Maximum: ${df['charges'].max():.2f}")
print(f"Skewness: {df['charges'].skew():.3f}")
print(f"Kurtosis: {df['charges'].kurtosis():.3f}")


## Task 3: Data Preprocessing and Feature Engineering


In [ ]:
# Data preprocessing and feature engineering
print("DATA PREPROCESSING")
print("=" * 30)

# Check for missing values
print("Missing values check:")
print(df.isnull().sum())

# Handle categorical variables using Label Encoding
le_gender = LabelEncoder()
le_smoker = LabelEncoder()
le_region = LabelEncoder()

df_processed = df.copy()
df_processed['gender_encoded'] = le_gender.fit_transform(df_processed['gender'])
df_processed['smoker_encoded'] = le_smoker.fit_transform(df_processed['smoker'])
df_processed['region_encoded'] = le_region.fit_transform(df_processed['region'])

print(f"\nCategorical encoding completed:")
print(f"Gender mapping: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")
print(f"Smoker mapping: {dict(zip(le_smoker.classes_, le_smoker.transform(le_smoker.classes_)))}")
print(f"Region mapping: {dict(zip(le_region.classes_, le_region.transform(le_region.classes_)))}")

# Outlier detection using Z-score
numerical_cols = ['age', 'bmi', 'children', 'charges']
z_scores = np.abs(stats.zscore(df_processed[numerical_cols]))
outlier_indices = np.where(z_scores > 3)
print(f"\nOutlier analysis:")
print(f"Number of outliers: {len(outlier_indices[0])}")

# Remove outliers
df_clean = df_processed[(z_scores < 3).all(axis=1)]
print(f"Dataset shape after outlier removal: {df_clean.shape}")

# Prepare features and target
feature_cols = ['age', 'bmi', 'children', 'gender_encoded', 'smoker_encoded', 'region_encoded']
X = df_clean[feature_cols]
y = df_clean['charges']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"Feature names: {feature_cols}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nScaled training set shape: {X_train_scaled.shape}")
print(f"Scaled test set shape: {X_test_scaled.shape}")


## Task 4: Linear Regression Model Implementation


In [ ]:
# Linear Regression Model Implementation
print("LINEAR REGRESSION MODEL TRAINING")
print("=" * 40)

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")
print(f"Model intercept: ${model.intercept_:.2f}")
print(f"Number of features: {len(model.coef_)}")

# Display feature coefficients
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nFeature Coefficients (sorted by absolute value):")
print(feature_importance)

# Make predictions
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Calculate regression metrics
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"\nModel Performance:")
print(f"Training MSE: ${mse_train:.2f}")
print(f"Test MSE: ${mse_test:.2f}")
print(f"Training R²: {r2_train:.4f}")
print(f"Test R²: {r2_test:.4f}")
print(f"Training MAE: ${mae_train:.2f}")
print(f"Test MAE: ${mae_test:.2f}")

# Calculate RMSE
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
print(f"Training RMSE: ${rmse_train:.2f}")
print(f"Test RMSE: ${rmse_test:.2f}")


## Task 5: Gradient Descent Implementation


In [ ]:
# Gradient Descent Implementation
print("GRADIENT DESCENT IMPLEMENTATION")
print("=" * 40)

# Initialize parameters
np.random.seed(42)
m, n = X_train_scaled.shape
theta = np.random.randn(n)  # Initial weights
bias = 0.0  # Initial bias
learning_rate = 0.01
epochs = 1000

print(f"Initial parameters:")
print(f"Theta shape: {theta.shape}")
print(f"Learning rate: {learning_rate}")
print(f"Number of epochs: {epochs}")

# Define the cost function (MSE)
def compute_cost(X, y, theta, bias):
    """Compute the mean squared error cost function"""
    m = len(y)
    predictions = np.dot(X, theta) + bias
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

# Define gradient descent function
def gradient_descent(X, y, theta, bias, learning_rate, epochs):
    """Perform gradient descent optimization"""
    m = len(y)
    cost_history = []
    
    for epoch in range(epochs):
        # Make predictions
        predictions = np.dot(X, theta) + bias
        
        # Compute gradients
        d_theta = (1 / m) * np.dot(X.T, (predictions - y))
        d_bias = (1 / m) * np.sum(predictions - y)
        
        # Update weights
        theta -= learning_rate * d_theta
        bias -= learning_rate * d_bias
        
        # Calculate cost and save it for plotting
        cost = compute_cost(X, y, theta, bias)
        cost_history.append(cost)
        
        # Print progress every 200 epochs
        if epoch % 200 == 0:
            print(f"Epoch {epoch}: Cost = ${cost:.2f}")
    
    return theta, bias, cost_history

print("\nStarting Gradient Descent Optimization...")
print("=" * 50)

# Run gradient descent
theta_gd, bias_gd, cost_history = gradient_descent(X_train_scaled, y_train, theta, bias, learning_rate, epochs)

print(f"\nGradient Descent Completed!")
print(f"Final theta: {theta_gd}")
print(f"Final bias: ${bias_gd:.2f}")
print(f"Final cost: ${cost_history[-1]:.2f}")

# Make predictions using gradient descent
y_pred_gd_train = np.dot(X_train_scaled, theta_gd) + bias_gd
y_pred_gd_test = np.dot(X_test_scaled, theta_gd) + bias_gd

# Calculate metrics for gradient descent
mse_gd_test = mean_squared_error(y_test, y_pred_gd_test)
r2_gd_test = r2_score(y_test, y_pred_gd_test)
mae_gd_test = mean_absolute_error(y_test, y_pred_gd_test)

print(f"\nGradient Descent Performance:")
print(f"Test MSE: ${mse_gd_test:.2f}")
print(f"Test R²: {r2_gd_test:.4f}")
print(f"Test MAE: ${mae_gd_test:.2f}")

# Compare with sklearn
print(f"\nComparison with Sklearn:")
print(f"Sklearn - MSE: ${mse_test:.2f}, R²: {r2_test:.4f}")
print(f"Gradient Descent - MSE: ${mse_gd_test:.2f}, R²: {r2_gd_test:.4f}")
print(f"Difference - MSE: ${abs(mse_test - mse_gd_test):.2f}, R²: {abs(r2_test - r2_gd_test):.4f}")


## Task 6: Classification Metrics Analysis


In [ ]:
# Classification Metrics Analysis
print("CLASSIFICATION METRICS ANALYSIS")
print("=" * 40)

# Convert continuous predictions to binary classifications for analysis
# Using median charges as threshold for high vs low insurance costs
threshold = y_test.median()
print(f"Classification threshold (median charges): ${threshold:.2f}")

# Convert actual charges to binary
y_test_binary = (y_test >= threshold).astype(int)
y_pred_binary = (y_pred_test >= threshold).astype(int)
y_pred_gd_binary = (y_pred_gd_test >= threshold).astype(int)

print(f"Class distribution in test set:")
print(f"Low charges (0): {(y_test_binary == 0).sum()} samples")
print(f"High charges (1): {(y_test_binary == 1).sum()} samples")

# Calculate classification metrics for sklearn model
accuracy_sklearn = accuracy_score(y_test_binary, y_pred_binary)
precision_sklearn = precision_score(y_test_binary, y_pred_binary)
recall_sklearn = recall_score(y_test_binary, y_pred_binary)
f1_sklearn = f1_score(y_test_binary, y_pred_binary)

# Calculate classification metrics for gradient descent model
accuracy_gd = accuracy_score(y_test_binary, y_pred_gd_binary)
precision_gd = precision_score(y_test_binary, y_pred_gd_binary)
recall_gd = recall_score(y_test_binary, y_pred_gd_binary)
f1_gd = f1_score(y_test_binary, y_pred_gd_binary)

print(f"\nSklearn Model Classification Metrics:")
print(f"Accuracy: {accuracy_sklearn:.4f}")
print(f"Precision: {precision_sklearn:.4f}")
print(f"Recall: {recall_sklearn:.4f}")
print(f"F1 Score: {f1_sklearn:.4f}")

print(f"\nGradient Descent Model Classification Metrics:")
print(f"Accuracy: {accuracy_gd:.4f}")
print(f"Precision: {precision_gd:.4f}")
print(f"Recall: {recall_gd:.4f}")
print(f"F1 Score: {f1_gd:.4f}")

# Confusion matrices
cm_sklearn = confusion_matrix(y_test_binary, y_pred_binary)
cm_gd = confusion_matrix(y_test_binary, y_pred_gd_binary)

print(f"\nSklearn Confusion Matrix:")
print(f"True Negatives: {cm_sklearn[0,0]}")
print(f"False Positives: {cm_sklearn[0,1]}")
print(f"False Negatives: {cm_sklearn[1,0]}")
print(f"True Positives: {cm_sklearn[1,1]}")

print(f"\nGradient Descent Confusion Matrix:")
print(f"True Negatives: {cm_gd[0,0]}")
print(f"False Positives: {cm_gd[0,1]}")
print(f"False Negatives: {cm_gd[1,0]}")
print(f"True Positives: {cm_gd[1,1]}")


## Task 7: Comprehensive Visualization and Analysis


In [ ]:
# Comprehensive Visualization Dashboard
fig = plt.figure(figsize=(24, 20))

# 1. Cost function over epochs (Gradient Descent)
plt.subplot(4, 5, 1)
plt.plot(range(epochs), cost_history, color='blue', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Cost (MSE)')
plt.title('Gradient Descent: Cost Function')
plt.grid(True, alpha=0.3)

# 2. Actual vs Predicted (Sklearn)
plt.subplot(4, 5, 2)
plt.scatter(y_test, y_pred_test, color='purple', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Charges ($)')
plt.ylabel('Predicted Charges ($)')
plt.title('Sklearn: Actual vs Predicted')
plt.grid(True, alpha=0.3)

# 3. Actual vs Predicted (Gradient Descent)
plt.subplot(4, 5, 3)
plt.scatter(y_test, y_pred_gd_test, color='green', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linewidth=2)
plt.xlabel('Actual Charges ($)')
plt.ylabel('Predicted Charges ($)')
plt.title('Gradient Descent: Actual vs Predicted')
plt.grid(True, alpha=0.3)

# 4. Residuals (Sklearn)
plt.subplot(4, 5, 4)
residuals_sklearn = y_test - y_pred_test
plt.scatter(y_pred_test, residuals_sklearn, alpha=0.6, color='purple')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values ($)')
plt.ylabel('Residuals ($)')
plt.title('Sklearn: Residuals vs Predicted')
plt.grid(True, alpha=0.3)

# 5. Residuals (Gradient Descent)
plt.subplot(4, 5, 5)
residuals_gd = y_test - y_pred_gd_test
plt.scatter(y_pred_gd_test, residuals_gd, alpha=0.6, color='green')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Values ($)')
plt.ylabel('Residuals ($)')
plt.title('Gradient Descent: Residuals vs Predicted')
plt.grid(True, alpha=0.3)

# 6. Feature Importance (Sklearn)
plt.subplot(4, 5, 6)
feature_importance_sklearn = feature_importance.sort_values('Coefficient', key=abs, ascending=True)
colors = ['red' if x < 0 else 'blue' for x in feature_importance_sklearn['Coefficient']]
bars = plt.barh(feature_importance_sklearn['Feature'], feature_importance_sklearn['Coefficient'], color=colors, alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Sklearn: Feature Importance')
plt.grid(True, alpha=0.3)

# 7. Feature Importance (Gradient Descent)
plt.subplot(4, 5, 7)
feature_importance_gd = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': theta_gd
}).sort_values('Coefficient', key=abs, ascending=True)
colors = ['red' if x < 0 else 'blue' for x in feature_importance_gd['Coefficient']]
bars = plt.barh(feature_importance_gd['Feature'], feature_importance_gd['Coefficient'], color=colors, alpha=0.7)
plt.xlabel('Coefficient Value')
plt.title('Gradient Descent: Feature Importance')
plt.grid(True, alpha=0.3)

# 8. Confusion Matrix (Sklearn)
plt.subplot(4, 5, 8)
sns.heatmap(cm_sklearn, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted Low', 'Predicted High'],
            yticklabels=['Actual Low', 'Actual High'])
plt.title('Sklearn: Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# 9. Confusion Matrix (Gradient Descent)
plt.subplot(4, 5, 9)
sns.heatmap(cm_gd, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Predicted Low', 'Predicted High'],
            yticklabels=['Actual Low', 'Actual High'])
plt.title('Gradient Descent: Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# 10. Classification Metrics Comparison
plt.subplot(4, 5, 10)
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
sklearn_values = [accuracy_sklearn, precision_sklearn, recall_sklearn, f1_sklearn]
gd_values = [accuracy_gd, precision_gd, recall_gd, f1_gd]

x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, sklearn_values, width, label='Sklearn', alpha=0.7, color='purple')
plt.bar(x + width/2, gd_values, width, label='Gradient Descent', alpha=0.7, color='green')

plt.xlabel('Metrics')
plt.ylabel('Score')
plt.title('Classification Metrics Comparison')
plt.xticks(x, metrics, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 11. ROC Curve (Sklearn)
plt.subplot(4, 5, 11)
fpr_sklearn, tpr_sklearn, _ = roc_curve(y_test_binary, y_pred_test)
roc_auc_sklearn = auc(fpr_sklearn, tpr_sklearn)
plt.plot(fpr_sklearn, tpr_sklearn, color='purple', lw=2, label=f'Sklearn (AUC = {roc_auc_sklearn:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Sklearn: ROC Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# 12. ROC Curve (Gradient Descent)
plt.subplot(4, 5, 12)
fpr_gd, tpr_gd, _ = roc_curve(y_test_binary, y_pred_gd_test)
roc_auc_gd = auc(fpr_gd, tpr_gd)
plt.plot(fpr_gd, tpr_gd, color='green', lw=2, label=f'Gradient Descent (AUC = {roc_auc_gd:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Gradient Descent: ROC Curve')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)

# 13. Prediction Distribution Comparison
plt.subplot(4, 5, 13)
plt.hist(y_pred_test, bins=30, alpha=0.7, color='purple', label='Sklearn', density=True)
plt.hist(y_pred_gd_test, bins=30, alpha=0.7, color='green', label='Gradient Descent', density=True)
plt.axvline(x=threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold (${threshold:.0f})')
plt.xlabel('Predicted Charges ($)')
plt.ylabel('Density')
plt.title('Prediction Distribution Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# 14. Error Analysis
plt.subplot(4, 5, 14)
errors_sklearn = np.abs(y_test - y_pred_test)
errors_gd = np.abs(y_test - y_pred_gd_test)
plt.scatter(y_pred_test, errors_sklearn, alpha=0.6, color='purple', label='Sklearn')
plt.scatter(y_pred_gd_test, errors_gd, alpha=0.6, color='green', label='Gradient Descent')
plt.xlabel('Predicted Value ($)')
plt.ylabel('Absolute Error ($)')
plt.title('Error Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

# 15. Model Performance Comparison
plt.subplot(4, 5, 15)
regression_metrics = ['MSE', 'R²', 'MAE', 'RMSE']
sklearn_reg_values = [mse_test, r2_test, mae_test, rmse_test]
gd_reg_values = [mse_gd_test, r2_gd_test, mae_gd_test, np.sqrt(mse_gd_test)]

x = np.arange(len(regression_metrics))
width = 0.35

plt.bar(x - width/2, sklearn_reg_values, width, label='Sklearn', alpha=0.7, color='purple')
plt.bar(x + width/2, gd_reg_values, width, label='Gradient Descent', alpha=0.7, color='green')

plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Regression Metrics Comparison')
plt.xticks(x, regression_metrics, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 16. Learning Curve (Cost vs Iterations)
plt.subplot(4, 5, 16)
plt.plot(range(100, epochs), cost_history[100:], color='blue', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Cost (MSE)')
plt.title('Learning Curve (Epochs 100-1000)')
plt.grid(True, alpha=0.3)

# 17. Cost Convergence
plt.subplot(4, 5, 17)
cost_diff = np.diff(cost_history)
plt.plot(range(1, len(cost_diff)+1), cost_diff, color='orange', linewidth=2)
plt.xlabel('Epochs')
plt.ylabel('Cost Change')
plt.title('Cost Convergence Rate')
plt.grid(True, alpha=0.3)

# 18. Residuals Distribution (Sklearn)
plt.subplot(4, 5, 18)
plt.hist(residuals_sklearn, bins=30, alpha=0.7, color='purple', edgecolor='black')
plt.xlabel('Residuals ($)')
plt.ylabel('Frequency')
plt.title('Sklearn: Residuals Distribution')
plt.grid(True, alpha=0.3)

# 19. Residuals Distribution (Gradient Descent)
plt.subplot(4, 5, 19)
plt.hist(residuals_gd, bins=30, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('Residuals ($)')
plt.ylabel('Frequency')
plt.title('Gradient Descent: Residuals Distribution')
plt.grid(True, alpha=0.3)

# 20. Model Comparison Summary
plt.subplot(4, 5, 20)
comparison_data = {
    'Model': ['Sklearn', 'Gradient Descent'],
    'MSE': [mse_test, mse_gd_test],
    'R²': [r2_test, r2_gd_test],
    'MAE': [mae_test, mae_gd_test],
    'Accuracy': [accuracy_sklearn, accuracy_gd]
}

# Create a simple comparison table
plt.text(0.1, 0.8, 'Model Performance Summary', fontsize=12, fontweight='bold', transform=plt.gca().transAxes)
plt.text(0.1, 0.7, f'Sklearn MSE: ${mse_test:.2f}', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.1, 0.6, f'Sklearn R²: {r2_test:.4f}', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.1, 0.5, f'GD MSE: ${mse_gd_test:.2f}', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.1, 0.4, f'GD R²: {r2_gd_test:.4f}', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.1, 0.3, f'Sklearn Acc: {accuracy_sklearn:.4f}', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.1, 0.2, f'GD Acc: {accuracy_gd:.4f}', fontsize=10, transform=plt.gca().transAxes)
plt.axis('off')
plt.title('Performance Summary')

plt.tight_layout()
plt.show()

# Final Summary
print("\n" + "="*60)
print("FINAL LAB 03 ANALYSIS SUMMARY")
print("="*60)
print(f"Dataset: Insurance charges prediction")
print(f"Total samples: {len(df)}")
print(f"Samples after preprocessing: {len(df_clean)}")
print(f"Features used: {len(feature_cols)}")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

print(f"\nSKLEARN MODEL PERFORMANCE:")
print(f"  Regression - MSE: ${mse_test:.2f}, R²: {r2_test:.4f}, MAE: ${mae_test:.2f}")
print(f"  Classification - Accuracy: {accuracy_sklearn:.4f}, F1: {f1_sklearn:.4f}")

print(f"\nGRADIENT DESCENT MODEL PERFORMANCE:")
print(f"  Regression - MSE: ${mse_gd_test:.2f}, R²: {r2_gd_test:.4f}, MAE: ${mae_gd_test:.2f}")
print(f"  Classification - Accuracy: {accuracy_gd:.4f}, F1: {f1_gd:.4f}")

print(f"\nKEY INSIGHTS:")
print(f"  • Smoking status is the most important predictor of insurance charges")
print(f"  • Age and BMI also significantly impact insurance costs")
print(f"  • Both models show similar performance, validating gradient descent implementation")
print(f"  • The model can predict insurance charges with reasonable accuracy")
print(f"  • Classification metrics help understand model performance for decision-making")
print("="*60)
